In [6]:
from pandas import DataFrame, read_csv
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# select different feature size and run cross validation
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from itertools import cycle
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import roc_auc_score

from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
    
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp


# read data and standardize
train_data = pd.read_csv("training_data.csv")
train_data_2 = train_data.iloc[:,1:]

X = train_data_2.iloc[:, :-1]
Y = train_data_2.iloc[:, -1]
X = preprocessing.scale(X)
# selece feature based on f_classif
X_new = SelectKBest(f_classif, k=100)
X_new.fit_transform(X, Y)
X = X[:, X_new.get_support()]
c = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
logreg = LogisticRegressionCV(penalty='l2', solver='sag', Cs=c, refit=True, cv=10, max_iter=100)
logreg.fit(X, Y)
score_list.append(logreg.score(X, Y))
# print(score_list[-1])    

print("The accuracy rate in training set is ", logreg.score(X, Y))


bagging = BaggingClassifier(KNeighborsClassifier(),
                            max_samples=0.5, max_features=0.5)


estimators = [("Tree", DecisionTreeRegressor()),
              ("Bagging(Tree)", BaggingRegressor(DecisionTreeRegressor()))]

bagging_2 = BaggingRegressor(DecisionTreeRegressor())                                   


Bagreg = bagging.fit(X, Y)
Bagreg.score(X, Y)
print("The accuracy rate in training set is (Bag_1)", Bagreg.score(X, Y))

Bagreg_2 = bagging_2.fit(X, Y)
Bagreg_2.score(X, Y)
print("The accuracy rate in training set is (Bag_2)", Bagreg_2.score(X, Y))



# test PCA at testing dataset
test_data = pd.read_csv("testing_data.csv")
test_data_2 = test_data.iloc[:,1:]

X_test = test_data_2.iloc[:, :-1]
Y_test = test_data_2.iloc[:, -1]
X_test = preprocessing.scale(X_test)
X_test = X_test[:, X_new.get_support()]

y_predict_1 = logreg.predict(X_test)
y_predict_2 = Bagreg.predict(X_test)
y_predict_3 = Bagreg_2.predict(X_test)


print("The AUC score is ", roc_auc_score(Y_test, y_predict_1))
print("The AUC score is ", roc_auc_score(Y_test, y_predict_2))
print("The AUC score is ", roc_auc_score(Y_test, y_predict_3))

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()


fpr[0], tpr[0], _ = roc_curve(Y_test, y_predict_1)
fpr[1], tpr[1], _ = roc_curve(Y_test, y_predict_2)
fpr[2], tpr[2], _ = roc_curve(Y_test, y_predict_3)


roc_auc[0] = auc(fpr[0], tpr[0])
roc_auc[1] = auc(fpr[1], tpr[1])
roc_auc[2] = auc(fpr[2], tpr[2])


plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr[0], tpr[0], label='Logistic Regression')
plt.plot(fpr[1], tpr[1], label='Ensemble methods-Bagging KNN')
plt.plot(fpr[2], tpr[2], label='Ensemble methods-Bagging Tree')

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

C:\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


The accuracy rate in training set is  0.718917217501
The accuracy rate in training set is (Bag_1) 0.734970097576
The accuracy rate in training set is (Bag_2) 0.826950393444
The AUC score is  0.598967313696
The AUC score is  0.591203651201
The AUC score is  0.647451727354
